In [1]:
import tabula
import sys
import pandas as pd
sys.path.append('../data')

In [2]:
sys.path.append('../scripts/python')
from clean_statements import Cleaner 
cleaner= Cleaner()

In [3]:
# convert PDF into CSV
tabula.convert_into("../data/Statement_All_Transactions_20220101_20220701.pdf", "../data/mpesaData.csv", output_format="csv", pages='all')

Got stderr: Sep 06, 2022 8:14:54 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Sep 06, 2022 8:14:54 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



In [4]:
summary= pd.read_csv('../data/mpesaData.csv', index_col = False,  nrows=11)
summary.drop('Unnamed: 3', axis=1, inplace=True)

In [5]:
summary

,TRANSACTION TYPE,PAID IN,PAID OUT
0,Cash Out,0.00,"7,390.00"
1,Send Money,"52,508.00","64,162.00"
2,Transaction Reversal,50.00,0.00
3,B2C Payment,"46,040.00",0.00
4,Pay Bill,0.00,"39,039.00"
5,Cash In,"51,000.00",0.00
6,FSI Withdraw,"17,000.00",0.00
7,FSI Deposit,0.00,"12,959.00"
8,Customer Merchant Payment,0.00,"39,717.50"
9,Customer Airtime Purchase,0.00,"3,630.00"


In [6]:
summary.to_csv('../data/financialSummary.csv', index=False)

In [7]:
Mpesa_data = pd.read_csv('../data/mpesaData.csv', index_col = False, skiprows=12)
Mpesa_data.drop('Unnamed: 7', axis=1, inplace=True)

In [8]:
Mpesa_data.head()

,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance
0,QG19OUVM6L,2022-07-01 18:25:24,Customer Transfer to 254798***904 -\rCaroline ...,COMPLETED,0.00,50.00,775.50
1,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill Charge,COMPLETED,0.00,23.00,825.50
2,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill to 200222 - NHIF Acc. 34664143,COMPLETED,0.00,500.00,848.50
3,QG10OHP9A0,2022-07-01 16:02:25,M-Shwari Withdraw,COMPLETED,"1,000.00",0.00,"1,348.50"
4,QG16OF8PA4,2022-07-01 15:31:39,Merchant Payment to 779959 - NAIVAS\rDEVELOPME...,COMPLETED,0.00,120.00,348.50


In [9]:
Mpesa_data['Transaction Status'].value_counts()

COMPLETED             660
Transaction Status     19
Name: Transaction Status, dtype: int64

1. Data Cleaning

Drop the Transaction status since all transactions were completed

In [10]:
Mpesa_data = pd.read_csv('../data/mpesaData.csv', index_col = False, skiprows=12)
Mpesa_data.drop('Unnamed: 7', axis=1, inplace=True)

In [11]:
Clean_data= cleaner.removerowsByValue(Mpesa_data, column='Transaction Status', val='Transaction Status')

In [12]:
Clean_data.tail()

,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance
673,QA516U45RJ,2022-01-05 18:58:30,Deposit of Funds at Agent Till 574553 -\rAMSEK...,COMPLETED,"15,000.00",0.00,"15,005.00"
675,QA516GVQ0H,2022-01-05 15:55:39,Buy Bundles,COMPLETED,0.00,20.00,5.00
676,QA5369RQA5,2022-01-05 13:56:44,Buy Bundles,COMPLETED,0.00,50.00,25.00
677,QA4552ABBB,2022-01-04 18:25:46,Merchant Payment to 7899629 -\rDORCAS CHEP...,COMPLETED,0.00,300.00,75.00
678,QA4552782V,2022-01-04 18:24:54,M-Shwari Withdraw,COMPLETED,200.00,0.00,375.00


In [19]:
Clean_data['Details'].unique

<bound method Series.unique of 0      Customer Transfer to 254798***904 -\rCaroline ...
1                                        Pay Bill Charge
2                Pay Bill to 200222 - NHIF Acc. 34664143
3                                      M-Shwari Withdraw
4      Merchant Payment to 779959 - NAIVAS\rDEVELOPME...
                             ...                        
673    Deposit of Funds at Agent Till 574553 -\rAMSEK...
675                                          Buy Bundles
676                                          Buy Bundles
677    Merchant Payment to 7899629 -\rDORCAS     CHEP...
678                                    M-Shwari Withdraw
Name: Details, Length: 660, dtype: object>

In [13]:
details=Clean_data['Details']

In [14]:
from unicodedata import category


def read_details(details):
    detsA=[]
    
    for row in details:
        if 'Pay Bill Online ' in row:
            category= 'Pay Bill Online'
        elif 'Customer Transfer' in row:
            category = 'Customer Transfer'
        elif 'Merchant Payment 'in row :
            category= 'Merchant Payment'
        elif 'Business Payment' in row:
            category = 'Business Payment' 
        elif 'Funds received' in row:
            category = 'Funds received'
        elif 'Customer Withdrawal' in row:
            category = 'Customer Withdrawal'
        elif 'Deposit of Funds' in row:
            category = 'Deposit of Funds'
        elif 'Buy Bundles' in row:
            category='Buy Bundles'
        elif 'M-Shwari Withdraw' in row:
            category= 'M-Shwari Withdraw'
        elif 'M-Shwari Deposit' in row:
            category= 'M-Shwari Deposit'
        elif 'Pay Bill Charge' in row:
            category= 'Pay Bill Charge'
        elif 'Airtime Purchase' in row:
            category= 'Airtime Purchase'
        elif 'Pay Bill to' in row:
            category= 'Pay Bill to'
        elif 'Pay Utility Reversal' in row:
            category = 'Pay Utility Reversal'
        elif 'Customer Payment to Small Business' in row:
            category= 'Payment to SME'
        else:
            category= row
        detsA.append(category)
    return detsA

In [15]:
dets= read_details(details)

In [16]:
Clean_data['Transaction Type']=dets

In [17]:
Clean_data.head(10)

,Receipt No,Completion Time,Details,Transaction Status,Paid in,Withdraw\rn,Balance,Transaction Type
0,QG19OUVM6L,2022-07-01 18:25:24,Customer Transfer to 254798***904 -\rCaroline ...,COMPLETED,0.00,50.00,775.50,Customer Transfer
1,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill Charge,COMPLETED,0.00,23.00,825.50,Pay Bill Charge
2,QG11OHV8NN,2022-07-01 16:04:31,Pay Bill to 200222 - NHIF Acc. 34664143,COMPLETED,0.00,500.00,848.50,Pay Bill to
3,QG10OHP9A0,2022-07-01 16:02:25,M-Shwari Withdraw,COMPLETED,"1,000.00",0.00,"1,348.50",M-Shwari Withdraw
4,QG16OF8PA4,2022-07-01 15:31:39,Merchant Payment to 779959 - NAIVAS\rDEVELOPME...,COMPLETED,0.00,120.00,348.50,Merchant Payment
5,QG13OAOXTX,2022-07-01 14:33:28,Customer Transfer to 254720***288 -\rMUSILI MUTUA,COMPLETED,0.00,50.00,468.50,Customer Transfer
6,QG16OAIJEQ,2022-07-01 14:31:18,M-Shwari Withdraw,COMPLETED,500.00,0.00,518.50,M-Shwari Withdraw
7,QG14NTN1JE,2022-07-01 10:49:40,Airtime Purchase,COMPLETED,0.00,100.00,18.50,Airtime Purchase
8,QG13NTLYMJ,2022-07-01 10:49:25,M-Shwari Withdraw,COMPLETED,100.00,0.00,118.50,M-Shwari Withdraw
9,QG17NHYJOZ,2022-07-01 07:53:43,Customer Transfer to 254718***386 -\rELIJAH MB...,COMPLETED,0.00,50.00,18.50,Customer Transfer


In [18]:
Data= cleaner.drop_columns(Clean_data,['Transaction Status','Details'])

In [19]:
Data.head()

,Receipt No,Completion Time,Paid in,Withdraw\rn,Balance,Transaction Type
0,QG19OUVM6L,2022-07-01 18:25:24,0.00,50.00,775.50,Customer Transfer
1,QG11OHV8NN,2022-07-01 16:04:31,0.00,23.00,825.50,Pay Bill Charge
2,QG11OHV8NN,2022-07-01 16:04:31,0.00,500.00,848.50,Pay Bill to
3,QG10OHP9A0,2022-07-01 16:02:25,"1,000.00",0.00,"1,348.50",M-Shwari Withdraw
4,QG16OF8PA4,2022-07-01 15:31:39,0.00,120.00,348.50,Merchant Payment


In [20]:
Data.dtypes

Receipt No          object
Completion Time     object
Paid in             object
Withdraw\rn         object
Balance             object
Transaction Type    object
dtype: object

Convert columns to the correct data types

In [21]:
Data['Paid in']= Data['Paid in'].str.replace(',', '').astype(float)

In [22]:
Data['Withdraw\rn']= Data['Withdraw\rn'].str.replace(',', '').astype(float)

In [23]:
Data['Balance']= Data['Balance'].str.replace(',', '').astype(float)

In [24]:
Data['Completion Time']=pd.to_datetime(Data['Completion Time'])

In [27]:
Data.dtypes

Receipt No                  object
Completion Time     datetime64[ns]
Paid in                    float64
Withdraw\rn                float64
Balance                    float64
Transaction Type            object
dtype: object

In [68]:
Data.columns

Index(['Receipt No', 'Completion Time', 'Paid in', 'Withdraw\rn', 'Balance',
       'Transaction Type'],
      dtype='object')

In [28]:
Data['Withdrawn']= Data ['Withdraw\rn']

In [29]:
Data.drop(columns='Withdraw\rn', inplace=True)

In [30]:
Data.head()

,Receipt No,Completion Time,Paid in,Balance,Transaction Type,Withdrawn
0,QG19OUVM6L,2022-07-01 18:25:24,0.0,775.5,Customer Transfer,50.0
1,QG11OHV8NN,2022-07-01 16:04:31,0.0,825.5,Pay Bill Charge,23.0
2,QG11OHV8NN,2022-07-01 16:04:31,0.0,848.5,Pay Bill to,500.0
3,QG10OHP9A0,2022-07-01 16:02:25,1000.0,1348.5,M-Shwari Withdraw,0.0
4,QG16OF8PA4,2022-07-01 15:31:39,0.0,348.5,Merchant Payment,120.0


In [31]:
Data.to_csv('../data/TransactionData.csv', index=False)